# 데이터 로드 및 전처리 과정 설명

1. **JSONL 파일에서 QA 쌍 로드**  
    - `data/qa_pairs.jsonl` 파일을 한 줄씩 읽어 각 줄을 JSON 객체로 파싱하여 `qa_pairs` 리스트에 저장합니다.
    - 전체 QA 쌍의 개수와 첫 번째 QA 쌍의 내용을 출력하여 데이터가 정상적으로 로드되었는지 확인합니다.

2. **QA 쌍 추가**  
    - 디버깅 및 테스트를 위해 두 개의 QA 쌍(`qa_add`)을 직접 리스트에 추가합니다.
    - `qa_pairs` 리스트에 이 데이터를 확장하여 추가합니다.

3. **데이터 확인**  
    - 전체 QA 쌍의 개수를 다시 출력하고, 마지막 두 개의 QA 쌍을 출력하여 추가된 데이터가 정상적으로 반영되었는지 확인합니다.

4. **파인튜닝용 데이터 변환 및 저장**  
    - 각 QA 쌍을 LLM 파인튜닝에 적합한 instruction 형태(`instruction`, `input`, `output` 필드)로 변환합니다.
    - 변환된 데이터를 `data/train_data.jsonl` 파일에 한 줄씩 저장합니다.  
    - `ensure_ascii=False` 옵션을 사용하여 한글이 깨지지 않도록 처리합니다.
    

In [ ]:
import json

# 파일 경로
file_path = "data/qa_pairs.jsonl"

# 파일 읽기
qa_pairs = []
with open(file_path, 'r', encoding='utf-8') as file:
    for line in file:
        # 각 줄을 JSON으로 파싱
        qa_pair = json.loads(line.strip())
        qa_pairs.append(qa_pair)

# 데이터 확인
print(f"총 {len(qa_pairs)}개의 QA 쌍을 로드했습니다.")
print("첫 번째 QA 쌍:", qa_pairs[0])

In [2]:
# 디버깅을 위한 추가
qa_add = [
    {
        "QUESTION": "전현상은 누구입니까?",
        "ANSWER": "전현상은 현재 Amazon Web Services의 Solutions Architect입니다.",
    },
    {
        "QUESTION": "금융보안교육센터의 주소 알려주세요.",
        "ANSWER": "금융보안교육센터의 주소는 서울특별시 영등포구 의사당대로 143 금투센터 8층 입니다.",
    },    
]
qa_pairs.extend(qa_add)

In [3]:


print(f"총 {len(qa_pairs)}개의 QA 쌍을 로드했습니다.")
print("마지막 두 개 QA 쌍:", qa_pairs[-2:])  # 새로 추가된 항목 확인

총 17개의 QA 쌍을 로드했습니다.
마지막 두 개 QA 쌍: [{'QUESTION': '전현상은 누구입니까?', 'ANSWER': '전현상은 현재 Amazon Web Services의 Solutions Architect입니다.'}, {'QUESTION': '금융보안교육센터의 주소 알려주세요.', 'ANSWER': '금융보안교육센터의 주소는 서울특별시 영등포구 의사당대로 143 금투센터 8층 입니다.'}]


# 파인튜닝을 위한 instruction 형태 변환
각 QA 쌍을 LLM 파인튜닝에 적합한 instruction 기반 포맷(`instruction`, `input`, `output` 필드)으로 변환합니다.  
변환된 데이터는 `train_data.jsonl` 파일에 한 줄씩 저장되며, 한글이 깨지지 않도록 `ensure_ascii=False` 옵션을 사용합니다.  
이 파일은 LLM 파인튜닝 학습에 사용됩니다.

In [4]:
import json

with open("data/train_data.jsonl", "w", encoding="utf-8") as f:
    for qa in qa_pairs:
        qa_modified = {
            "instruction": qa["QUESTION"],
            "input": "",
            "output": qa["ANSWER"],
        }
        f.write(json.dumps(qa_modified, ensure_ascii=False) + "\n")

# 데이터 분석 및 시각화(옵션)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from collections import Counter

# 질문/답변 길이 분석
q_lengths = [len(qa["QUESTION"]) for qa in filtered_qa_pairs]
a_lengths = [len(qa["ANSWER"]) for qa in filtered_qa_pairs]

# 기본 통계 계산
print(f"질문 평균 길이: {np.mean(q_lengths):.1f} 문자 (최소: {min(q_lengths)}, 최대: {max(q_lengths)})")
print(f"답변 평균 길이: {np.mean(a_lengths):.1f} 문자 (최소: {min(a_lengths)}, 최대: {max(a_lengths)})")

# 시각화
plt.figure(figsize=(15, 5))

# 질문 길이 분포
plt.subplot(1, 2, 1)
plt.hist(q_lengths, bins=20, alpha=0.7)
plt.title('질문 길이 분포')
plt.xlabel('문자 길이')
plt.ylabel('빈도')

# 답변 길이 분포
plt.subplot(1, 2, 2)
plt.hist(a_lengths, bins=20, alpha=0.7)
plt.title('답변 길이 분포')
plt.xlabel('문자 길이')
plt.ylabel('빈도')

plt.tight_layout()
plt.show()